In [1]:
import os
import json
from collections import OrderedDict

from evaluate_lanenet_on_tusimple import eval_lanenet
from modules.homo_util import homoify
from modules.video_util import videoify

In [2]:
DATA_PATH = "../test_set"
JSON_PATH = "outputs/json/"
SAVE_DIR = "outputs/seg_results/"
VID_DIR = "outputs/vids/"

CONST_IMAGE = (1312, 1312)
LANENET_WEIGHTS="weights/tusimple_lanenet.ckpt"

In [3]:
# frames_dir = "../test_set/clips/0531/1492626499813320696"
frames_dir = "../test_set/clips/0530/1492626191132352208_0"

In [4]:
clip_num = "_".join(frames_dir.split("/")[3:])
save_json = os.path.join(JSON_PATH, clip_num + ".json")
vid_path = os.path.join(VID_DIR, clip_num + ".mp4")
clip_num

'0530_1492626191132352208_0'

Detecting Lanes using lanenet

In [5]:
print("Detecting lanes")
lanes_json = eval_lanenet(frames_dir, LANENET_WEIGHTS, SAVE_DIR, save_json)
print("Lanes detected")
lanes_json = OrderedDict(sorted(lanes_json.items(), key=lambda x: int(x[0].split("/")[-1].split(".")[0])))

Detecting lanes


100%|██████████| 20/20 [00:10<00:00,  1.95it/s]

Lanes detected


In [6]:
lanes_json.keys()

odict_keys(['clips/0530/1492626191132352208_0/1.jpg', 'clips/0530/1492626191132352208_0/2.jpg', 'clips/0530/1492626191132352208_0/3.jpg', 'clips/0530/1492626191132352208_0/4.jpg', 'clips/0530/1492626191132352208_0/5.jpg', 'clips/0530/1492626191132352208_0/6.jpg', 'clips/0530/1492626191132352208_0/7.jpg', 'clips/0530/1492626191132352208_0/8.jpg', 'clips/0530/1492626191132352208_0/9.jpg', 'clips/0530/1492626191132352208_0/10.jpg', 'clips/0530/1492626191132352208_0/11.jpg', 'clips/0530/1492626191132352208_0/12.jpg', 'clips/0530/1492626191132352208_0/13.jpg', 'clips/0530/1492626191132352208_0/14.jpg', 'clips/0530/1492626191132352208_0/15.jpg', 'clips/0530/1492626191132352208_0/16.jpg', 'clips/0530/1492626191132352208_0/17.jpg', 'clips/0530/1492626191132352208_0/18.jpg', 'clips/0530/1492626191132352208_0/19.jpg', 'clips/0530/1492626191132352208_0/20.jpg'])

Yolo

In [7]:
frames_dir

'../test_set/clips/0530/1492626191132352208_0'

In [8]:
YOLO_WEIGHTS = "../yolov7/yolov7.pt"
YOLO_PATH = "outputs/yolo"
CONF = 0.25
SOURCE = frames_dir
IMG_SIZE = 1280

In [9]:
os.system(f"python ../yolov7/detect.py --weights {YOLO_WEIGHTS} --source {SOURCE} --conf {CONF} --img-size {IMG_SIZE} --save-txt --project {YOLO_PATH}")
print("YOLO done")

YOLO done


Homography

In [10]:
final_frames = homoify(lanes_json, DATA_PATH, YOLO_PATH, CONST_IMAGE)
print("Final Frames ready for video writing")

Final Frames ready for video writing


Writing Video

In [11]:
videoify(final_frames, vid_path)
print(f"Video written in {vid_path}")

Video written in outputs/vids/0530_1492626191132352208_0.mp4
